# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [44]:
# import numpy and pandas
import pandas as pd
import numpy as np
from scipy.stats import trim_mean, mode, skew, gaussian_kde, pearsonr, spearmanr, beta
from statsmodels.stats.weightstats import ztest as ztest

from scipy.stats import ttest_ind, norm, t
from scipy.stats import f_oneway
from scipy.stats import sem

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [45]:
# Run this code:
salaries = pd.read_csv(r'C:\DSML bootcamp\Week4\Day3\lab-hypothesis-testing-en\data\Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [46]:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [47]:
from scipy.stats import ttest_1samp

In [48]:
#  Filter only hourly employees from the dataset
hourly_wages = salaries[salaries["Salary or Hourly"] == "Hourly"]["Hourly Rate"]

#  Remove any missing values to avoid errors in calculations
hourly_wages = hourly_wages.dropna()

hourly_wages

4        19.86
6        46.10
7        35.60
10        2.65
18       17.68
         ...  
33164    46.10
33168    17.68
33169    35.60
33174    46.35
33175    48.85
Name: Hourly Rate, Length: 8022, dtype: float64

In [49]:
# Perform a one-sample t-test comparing mean hourly wage to $30
t_stat, p_value = ttest_1samp(hourly_wages, 30)

# Print the t-statistic (how far the sample mean is from $30 in SE units)
print(f"T-statistic: {t_stat}")

#  Print the p-value (probability of observing the data if H0 is true)
print(f"P-value: {p_value}")

# Set significance level alpha = 0.05 (for 95% confidence)
alpha = 0.05

#  Decision rule: if p-value < alpha → reject H0
if p_value < alpha:
    print("Reject the null hypothesis — mean hourly wage is significantly different from $30/hr.")
else:
    print("Fail to reject the null hypothesis — no significant difference from $30/hr.")

T-statistic: 20.6198057854942
P-value: 4.3230240486229894e-92
Reject the null hypothesis — mean hourly wage is significantly different from $30/hr.


# Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [58]:
# Set the desired confidence level (95% means we are 95% confident the interval contains the true mean)
confidence_level = 0.95  

#  = sample size - 1, needed for t-distribution
degrees_freedom = len(hourly_workers_df) - 1 

# Calculate the mean hourly rate from the dataset
sample_mean = hourly_workers_df['Hourly Rate'].mean()  

# Calculate the standard error of the mean using SciPy's sem() function
standard_error = sem(hourly_workers_df['Hourly Rate'])  

# Compute the confidence interval using Student's t-distribution
# Parameters:
# confidence = confidence level
# df = degrees_freedom
# loc = sample mean
# scale = standard error
CI = t.interval(confidence=confidence_level, df=degrees_freedom, loc=sample_mean, scale=standard_error)   

print(CI)
print(f"The 95% confidence interval for the mean hourly wage is: ${CI[0]} to ${CI[1]}")  


(np.float64(32.52345834488425), np.float64(33.05365708767623))
The 95% confidence interval for the mean hourly wage is: $32.52345834488425 to $33.05365708767623


# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [65]:
# Import the z-test function for proportions from the statsmodels library
from statsmodels.stats.proportion import proportions_ztest  

In [72]:
# Step 1: Count hourly workers
count_hourly = sum(salaries['Salary or Hourly'] == 'Hourly')

# Step 2: Get total number of employees
total_employees = len(salaries)

# Step 3: Set the null hypothesis proportion (p0)
# H0: proportion of hourly workers = 0.25 (25%)
p0 = 0.25

# Step 4: Run the proportions z-test
# count = number of "successes" (hourly workers)
# nobs = total observations (total employees)
z_stat, p_value = proportions_ztest(count=count_hourly, nobs=total_employees, value=p0)

# Step 5: Print results
print("Z-statistic:", z_stat)
print("P-value:", p_value)

# Step 6: Interpret results at 95% confidence
if p_value < 0.05:
    print("We reject the null hypothesis: the proportion of hourly workers is significantly different from 25%.")
else:
    print("We fail to reject the null hypothesis: no significant difference from 25%.")

Z-statistic: -3.5099964213703005
P-value: 0.0004481127249057967
We reject the null hypothesis: the proportion of hourly workers is significantly different from 25%.
